In [ ]:
import tkinter as tk                              # Tkinter für GUI
import serial                                   # pyserial für serielle Kommunikation
import re                                       # Reguläre Ausdrücke zum Parsen
import matplotlib.pyplot as plt                 # Matplotlib zum Erstellen der Diagramme
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg  # Integration von Matplotlib in Tkinter

# UART-Verbindung erstellen (Port COM7, 9600 Baud)
ser = serial.Serial(
    'COM7',
    baudrate=9600,
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_EVEN,
    stopbits=serial.STOPBITS_ONE,
    timeout=1
)
                

# Datenarrays für das Abstandsmessdiagramm
time_values = []                                 # Liste der Zeitpunkte (einfache Zähler)
position_values = []                             # Liste der Abstandswerte
counter = 0                                      # Einfacher Zähler für "Zeit"

# Neue Arrays für die Motorpositionsdaten
motor_x_history = []                             # Speichert Motor X-Werte 
motor_y_history = []                             # Speichert Motor Y-Werte
motor_z_history = []                             # Speichert Motor Z-Werte

messung_aktiv = 0                               # Flag, ob Abstandsmessung aktiv ist
pos_aktiv = 0

def toggle_measurement(*args):
    global counter, messung_aktiv                # benötigte globale Variablen
    """Sendet 'M' beim Aktivieren und 'MN' beim Deaktivieren"""
    if abstand_var.get():
        ser.write("M\n".encode())               # Sende 'M' zum Start der Messung
        messung_aktiv = 1                        # Setze Messung aktiv
    else:
        ser.write("MN\n".encode())              # Sende 'MN' zum Stoppen der Messung
        messung_aktiv = 0                        # Setze Messung inaktiv
        ser.reset_input_buffer()                 # Leere den Empfangspuffer

def send_text():
    zeile1 = text_line1.get().strip()
    zeile2 = text_line2.get().strip()
    if ser and ser.is_open:
        if zeile1:
            ser.write(f"{zeile1}:Texto\n".encode())
        if zeile2:
            ser.write(f"{zeile2}:Textu\n".encode())
        else:
            print("Kein Text gesendet")              # Andernfalls Meldung ausgeben

def toggle_ref(*args):
    if ref_var.get():
        ser.write("R\n".encode())               # Sende 'R' zum Starten der Referenzfahrt
    else:
        print("keine Referenzfahrt")            # Ausgabe falls nicht aktiviert

def start_action():
    """Sendet Start-Signal an Arduino"""
    ser.write("START\n".encode())               # Sende 'START'

def stop_action():
    """Sendet Not-Aus-Signal an Arduino"""
    ser.write("STOP\n".encode())                # Sende 'STOP'
    
def toggle_pos(*args):
    global counter, pos_aktiv                # benötigte globale Variablen
    """Sendet 'POS' beim Aktivieren und 'NPOS' beim Deaktivieren"""
    if pos_var.get():
        ser.write("POS\n".encode())               # Sende 'M' zum Start der Messung
        pos_aktiv = 1                        # Setze Messung aktiv
    else:
        ser.write("NPOS\n".encode())              # Sende 'MN' zum Stoppen der Messung
        pos_aktiv = 0                        # Setze Messung inaktiv
        ser.reset_input_buffer()                 # Leere den Empfangspuffer

def update_plot():
    """Aktualisiert das Diagramm für Abstandsmessung über Zeit"""
    ax.clear()                                  # Löscht den aktuellen Plot von Abstandsmessung
    ax.plot(time_values, position_values, marker="x", linestyle="-", color="red")  # Zeichnet den neuen Plot
    ax.set_title("Abstandsmessung über Zeit")    # Setzt Titel
    ax.set_xlabel("Zeit")                        # Beschriftet X-Achse
    ax.set_ylabel("Position")                    # Beschriftet Y-Achse
    canvas.draw()                                # Aktualisiert die Anzeige des Diagramms

def update_motor_plots():
    """Aktualisiert die beiden Diagramme für Motorpositionen"""
    # Diagramm 1: Motor X (Y-Achse) vs Motor Z (X-Achse)
    ax_motor1.clear()                           # Löscht Diagramm 1
    ax_motor1.plot(motor_z_history, motor_x_history, marker="o", linestyle="-", color="blue")
    ax_motor1.set_title("Motor X vs Motor Z")    # Titel des Diagramms
    ax_motor1.set_xlabel("Motor Z")              # Achsenbeschriftung (X-Achse)
    ax_motor1.set_ylabel("Motor X")              # Achsenbeschriftung (Y-Achse)
    canvas_motor1.draw()                         # Aktualisiert Diagramm 1

    # Diagramm 2: Motor X (Y-Achse) vs Motor Y (X-Achse)
    ax_motor2.clear()                           # Löscht Diagramm 2
    ax_motor2.plot(motor_y_history, motor_x_history, marker="o", linestyle="-", color="green")
    ax_motor2.set_title("Motor X vs Motor Y")    # Titel des Diagramms
    ax_motor2.set_xlabel("Motor Y")              # X-Achse: Motor Y
    ax_motor2.set_ylabel("Motor X")              # Y-Achse: Motor X
    canvas_motor2.draw()                         # Aktualisiert Diagramm 2

def receive_data():
    """
    Liest alle vollständigen Zeilen vom µC und verarbeitet sie:
      - Abstand (enthält 'mm' und messung_aktiv)
      - Motorposition (beginnt mit 'X:' und pos_aktiv)
      - Sonstige Ausgaben (Debug, Buchstaben, POS/NPOS, …)
    """
    global counter

    try:
        # Solange eine komplette Zeile im Puffer steht, lese sie
        while True:
            raw = ser.readline()               # blockiert bis '\n' oder Timeout
            if not raw:
                break

            line = raw.decode("latin-1", errors="ignore").strip()
            if not line:
                continue

            # 1) Abstandsmessung
            if messung_aktiv and "mm" in line:
                m = re.search(r"\d+", line)
                if m:
                    d = int(m.group())
                    time_values.append(counter)
                    position_values.append(d)
                    counter += 1
                    update_plot()
                else:
                    print("Ungültiger Abstand:", line)

            # 2) Motorposition
            elif pos_aktiv and line.startswith("X:"):
                found = False
                for m in re.finditer(
                    r"X[:=]\s*(\d+)\s*"
                    r"Y[:=]\s*(\d+)\s*"
                    r"Z[:=]\s*(\d+)", line
                ):
                    x, y, z = map(int, m.groups())
                    motor_x_history.append(x)
                    motor_y_history.append(y)
                    motor_z_history.append(z)
                    found = True
                if found:
                    update_motor_plots()
                else:
                    print("Ungültige POS-Zeile:", line)

            # 3) Alles andere
            else:
                print(line)

    except Exception as e:
        print("Fehler beim Lesen:", e)

    finally:
        # Wiederhole alle 100 ms
        root.after(100, receive_data)

# Hauptfenster erstellen
root = tk.Tk()                              
root.title("Arduino Steuerung")

# Oberer Bereich für Checkboxen, Buttons u. Eingabefeld
top_frame = tk.Frame(root)
top_frame.pack(side=tk.TOP, fill=tk.X, padx=10, pady=10)

# Checkbox für Referenzfahrt
ref_var = tk.BooleanVar()  # Variable für den Zustand der Referenzfahrt
ref_checkbox = tk.Checkbutton(top_frame, text="Referenzfahrt", variable=ref_var)
ref_checkbox.pack(side=tk.LEFT, padx=5)
ref_var.trace_add("write", toggle_ref)

# Checkbox für Abstand messen
abstand_var = tk.BooleanVar()  # Variable für die Abstandsmessung
abstand_checkbox = tk.Checkbutton(top_frame, text="Abstand messen", variable=abstand_var)
abstand_checkbox.pack(side=tk.LEFT, padx=5)
abstand_var.trace_add("write", toggle_measurement)

# Checkbox für Positionierung
pos_var = tk.BooleanVar()  # Variable für die Positionierung
pos_checkbox = tk.Checkbutton(top_frame, text="Positionierung", variable=pos_var)
pos_checkbox.pack(side=tk.LEFT, padx=5)
pos_var.trace_add("write", toggle_pos)

# Neues Frame für zweizeilige Eingabe
text_frame = tk.Frame(root)
text_frame.pack(side=tk.TOP, fill=tk.X, padx=10, pady=5)

label1 = tk.Label(text_frame, text="Zeile 1:")
label1.pack(anchor="w")
text_line1 = tk.Entry(text_frame)
text_line1.pack(fill=tk.X, padx=5, pady=2)

label2 = tk.Label(text_frame, text="Zeile 2:")
label2.pack(anchor="w")
text_line2 = tk.Entry(text_frame)
text_line2.pack(fill=tk.X, padx=5, pady=2)

# Sende-Button separat platzieren
send_button = tk.Button(text_frame, text="Sende Text", command=send_text)
send_button.pack(pady=5)


# Buttons "Start" und "Not-Aus" in einem weiteren Frame
button_frame = tk.Frame(root)
button_frame.pack(side=tk.TOP, fill=tk.X, padx=10, pady=5)

start_button = tk.Button(button_frame, text="Start", command=start_action)
start_button.pack(side=tk.LEFT, padx=5)
stop_button = tk.Button(button_frame, text="Not-Aus", command=stop_action, bg="red", fg="white")
stop_button.pack(side=tk.LEFT, padx=5)

diagram_frame = tk.Frame(root)
diagram_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=10)

# Grid-Konfiguration: eine Zeile, drei Spalten, alle gleich gewichtet
diagram_frame.grid_rowconfigure(0, weight=1)
for col in range(3):
    diagram_frame.grid_columnconfigure(col, weight=1)

# Diagramm 1: Abstandsmessung
fig, ax = plt.subplots()
canvas = FigureCanvasTkAgg(fig, master=diagram_frame)
widget1 = canvas.get_tk_widget()
widget1.grid(row=0, column=0, sticky="nsew", padx=5, pady=5)

# Diagramm 2: Motor X vs Motor Z
fig_motor1, ax_motor1 = plt.subplots()
canvas_motor1 = FigureCanvasTkAgg(fig_motor1, master=diagram_frame)
widget2 = canvas_motor1.get_tk_widget()
widget2.grid(row=0, column=1, sticky="nsew", padx=5, pady=5)

# Diagramm 3: Motor X vs Motor Y
fig_motor2, ax_motor2 = plt.subplots()
canvas_motor2 = FigureCanvasTkAgg(fig_motor2, master=diagram_frame)
widget3 = canvas_motor2.get_tk_widget()
widget3.grid(row=0, column=2, sticky="nsew", padx=5, pady=5)


# Starte die Datenempfangsfunktion (wird alle 100 ms aufgerufen)
root.after(100, receive_data)

root.mainloop()

Referenzfahrt gestartet.
keine Referenzfahrt
POS
Zeile 1 wird geschrieben:
ABC             Zeile 2 wird geschrieben:
DEF
Wird gestartet.
ABC
DEF
